In [1]:
import requests
import bs4
import json

In [71]:
max_pages = 0
current_page = 0
typology_index = 0

url_base = 'https://collections.louvre.fr/en/recherche?page=1&typology%5B0%5D=1'
url_scrap = f'https://collections.louvre.fr/en/recherche?page={current_page}&typology[{typology_index}]=1'
url_art_DB_base = 'https://collections.louvre.fr'



#typology = {'Peintures' : f'Typology[{typology_index}]=1'}

In [86]:
# DOM content
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
# https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent


def get_weblimits(url):
    response = requests.get(url_base, headers=headers) #, timeout=0.2)
    html = response.content
    print(f'Status code is: {response.status_code}')

    response = requests.get(url_base, headers=headers) #, timeout=0.05)
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")

    span = soup.find('input', class_='pagination__current nav__pagination__nbr no_style')
    max_number_of_pages = int(span['max'])
    current_page = int(span['value'])

    print(f'There are {max_number_of_pages} pages to be scraped.\n')

    return max_number_of_pages, current_page


def get_artworks_links(url):

    link_list = []

    response = requests.get(url, headers=headers) #, timeout=0.2)
    html = response.content
    print(f'Status code is: {response.status_code}')

    response = requests.get(url, headers=headers) #, timeout=0.05)
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")

    span = soup.findAll('a', class_='card--search card__link')

    for i in range(0, len(span)):
        link_list.append(span[i]['href'])

    print(f'There are {len(link_list)} items in this page.\n')    
    return link_list
    
def get_metada_thumbail(links):

    for i in links:
       
        metadata = requests.get(url_art_DB_base+ i +'.json').content
        print(f'./json/{i[-11:]}')

        print(json.loads(metadata)['image'][0]['urlThumbnail'])
        thumbnail = requests.get(json.loads(metadata)['image'][0]['urlThumbnail']).content

        with open(f'./art_pieces_json/{i[-11:]}.json', 'wb') as f:
            f.write(metadata)  

        with open(f'./art_pieces_thumbnails/images_thumbnails/{i[-11:]}.jpg', 'wb') as f:
            f.write(thumbnail)

In [87]:
max_pages, current_page = get_weblimits(url_base)

# 10 webs / 1.5 min

for i in range(current_page, 11): #max_pages
    
    url_scrap = f'https://collections.louvre.fr/en/recherche?page={i}&typology[{typology_index}]=1'
    print(f'Scanning page {i} of {max_pages}')
    print(url_scrap, '\n')

    links = get_artworks_links(url_scrap)
    print(len(links))
    get_metada_thumbail(links)
    

Status code is: 200
There are 511 pages to be scraped.

Scanning page 1 of 511
https://collections.louvre.fr/en/recherche?page=1&typology[0]=1 

Status code is: 200
There are 20 items in this page.

20
./json/cl010052688
https://collections.louvre.fr/media/cache/small/0000000021/0000052688/0000041267_OG.JPG
./json/cl010052692
https://collections.louvre.fr/media/cache/small/0000000021/0000052692/0001063753_OG.JPG
./json/cl010052695
https://collections.louvre.fr/media/cache/small/0000000021/0000052695/0000968477_OG.JPG
./json/cl010052698
https://collections.louvre.fr/media/cache/small/0000000021/0000052698/0000972835_OG.JPG
./json/cl010052702
https://collections.louvre.fr/media/cache/small/0000000021/0000052702/0000924957_OG.JPG
./json/cl010052712
https://collections.louvre.fr/media/cache/small/0000000021/0000052712/0000962880_OG.JPG
./json/cl010052768
https://collections.louvre.fr/media/cache/small/0000000021/0000052768/0000886264_OG.JPG
./json/cl010052777
https://collections.louvre.fr/